In [1]:
#default_exp sklearn

In [2]:
#export
from hypster.oo_hp import *
from hypster.hypster_prepare import *

import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

from sklearn.model_selection import train_test_split

from copy import deepcopy

import optuna

In [3]:
#export
SEED = 42

# Read Data

In [4]:
#export
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('C:/Users/user/.fastai/data/adult_sample/adult.csv'),Path('C:/Users/user/.fastai/data/adult_sample/export.pkl'),Path('C:/Users/user/.fastai/data/adult_sample/models')]

In [5]:
#export
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [6]:
#export
df = df.sample(frac=0.1)

In [7]:
#export
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
dep_var = "salary"

In [8]:
y = np.where(df[dep_var] == ">=50k", 1, 0)
df.drop(dep_var, axis=1, inplace=True)

In [9]:
#export
train_df, test_df, y_train, y_test = train_test_split(df, y, test_size=0.6, 
                                                     random_state=SEED, 
                                                     stratify=y)

# Preprocessing

In [10]:
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
31699,32,Private,206051,Some-college,10.0,Never-married,Adm-clerical,Not-in-family,White,Male,0,0,35,United-States
15667,45,Local-gov,37306,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,United-States
15906,34,Self-emp-not-inc,49707,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,70,United-States
16360,30,Private,167832,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
22636,42,Federal-gov,262402,Bachelors,13.0,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,40,United-States


In [11]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

In [12]:
from sklearn.impute import SimpleImputer

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
imputer = SimpleImputer()

In [16]:
RandomForestClassifier = prepare(RandomForestClassifier)
Pipeline = prepare(Pipeline)

In [17]:
rf = RandomForestClassifier(n_estimators=HpInt("n_trees", 10, 40, 10),
                            criterion=HpCategorical("criterion", ['gini', 'entropy']),
                            max_depth=HpToggle("max_depth_toggle", HpInt("max_depth", 5, 30, 5)),
                            max_features=HpCategorical("max_features", ['auto', 'sqrt', 'log2', None]),
                            max_leaf_nodes=HpToggle("max_leaf_nodes_tog", HpInt("max_leaf_nodes", 2, 20)),
                            min_impurity_decrease=0.0,
                            bootstrap=HpBool("bootstrap"),
                            oob_score=False,
                            n_jobs=-1,
                            random_state=SEED,
                            verbose=0,
                            warm_start=False,
                            class_weight=None,
                            ccp_alpha=0.0,
                            max_samples=None
                            )

In [18]:
from category_encoders import BinaryEncoder

In [19]:
cat_enc = BinaryEncoder()

In [20]:
cont_imp  = ColumnTransformer([("imp", imputer, cont_names)])

In [21]:
cat_enc   = ColumnTransformer([("cat_enc", cat_enc, cat_names)])

In [22]:
fe = FeatureUnion([("cat", cat_enc), ("cont", cont_imp)])

In [23]:
pipe = Pipeline([("fe", fe), ("model", rf)])

In [24]:
#export
import datetime
def run_learner(fit_method, get_metric, n_trials=5): #learner
    class Objective():
        def __init__(self, fit_method, get_metric): #learner
            #self.learner   = learner
            self.fit_method = fit_method
            self.get_metric = get_metric
            
        def __call__(self, trial): 
            #learner = self.learner.sample(trial)
            self.fit_method.sample(trial)
            res = self.get_metric.sample(trial)
            #print(self.fit_method.base_call)
            #print(self.get_metric.base_call.base_call)
            print(res)
            return res

    objective = Objective(fit_method, get_metric) #learner
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    study = optuna.create_study(direction="maximize", study_name = now, pruner=pruner)
    study.optimize(objective, n_trials=n_trials, n_jobs=1, timeout=600)
    return study

In [25]:
#export
study = run_learner(#learner    = learner,
                    fit_method = pipe.fit(train_df, y_train),
                    get_metric = pipe.score(test_df, y_test),
                    n_trials   = 3
                   )

0.8004094165813715
0.8382804503582395
0.7722620266120778


In [26]:
#export
print("Number of finished trials: {}".format(len(study.trials)))

Number of finished trials: 3


In [27]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

Best trial:
  Value: 0.8382804503582395


In [28]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Params: 
    n_trees: 20
    criterion: gini
    max_depth_toggle: False
    max_features: sqrt
    max_leaf_nodes_tog: True
    max_leaf_nodes: 12
    bootstrap: False


In [29]:
#export
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,params_bootstrap,params_criterion,params_max_depth,params_max_depth_toggle,params_max_features,params_max_leaf_nodes,params_max_leaf_nodes_tog,params_n_trees,state
0,0,0.800409,2020-05-30 16:46:34.500328,2020-05-30 16:46:34.868798,True,gini,25.0,True,sqrt,4.0,True,30,COMPLETE
1,1,0.838280,2020-05-30 16:46:34.869766,2020-05-30 16:46:35.226690,False,gini,NaN,False,sqrt,12.0,True,20,COMPLETE
2,2,0.772262,2020-05-30 16:46:35.227720,2020-05-30 16:46:35.585253,False,gini,NaN,False,None,NaN,False,10,COMPLETE
